In [1]:
import pandas as pd
import MySQLdb
import os
import time

In [130]:
db = MySQLdb.connect("47.100.14.137","hyps44413","Nanhua")
db.set_character_set('gbk')

In [128]:
db = MySQLdb.connect("localhost","root","123456")
db.set_character_set("utf8")

# 原油

In [3]:
DATA_DIR = "../CL Daily Data WTI原油期货-所有资料"
for filename in os.listdir(DATA_DIR):
      
    cursor = db.cursor()
    sql = "CREATE DATABASE IF NOT EXISTS WTI_Oil_original_data"
    cursor.execute(sql)
    
    contract = filename.split(".xlsx")[0]
    
    sql = "CREATE TABLE IF NOT EXISTS WTI_Oil_original_data." + contract + \
          " (Date DATE PRIMARY KEY,\
            Open FLOAT(5),\
            High FLOAT(5),\
            Low FLOAT(5),\
            Close FLOAT(5),\
            Volume FLOAT(5),\
            OI FLOAT(5))"
    cursor.execute(sql)
    
    
    data = pd.read_excel(DATA_DIR+"/"+filename,header=None)
    data = data.fillna("NULL")
    
    
    for i in range(8,len(data)):
        cursor = db.cursor()
        sql = "insert into WTI_Oil_original_data." + contract + " values ('"+ data.iloc[i][0].strftime("%Y-%m-%d") + "',"\
                + str(data.iloc[i][1]) + ","\
                + str(data.iloc[i][2]) + ","\
                + str(data.iloc[i][3]) + ","\
                + str(data.iloc[i][4]) + ","\
                + str(data.iloc[i][5]) + ","\
                + str(data.iloc[i][6]) + ")"
        index = i
        cursor.execute(sql)
    cursor.close()

C:\Users\Philip\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: Warning: (1007, "Can't create database 'wti_oil_original_data'; database exists")
  


# 鐵礦石

In [4]:
DATA_DIR = "../all IOE"
for filename in os.listdir(DATA_DIR):
      
    cursor = db.cursor()
    sql = "CREATE DATABASE IF NOT EXISTS Iron_original_data"
    cursor.execute(sql)
    
    contract = filename.split(".xlsx")[0]
    
    sql = "CREATE TABLE IF NOT EXISTS Iron_original_data." + contract + \
          " (Date DATE,\
            Open FLOAT,\
            High FLOAT,\
            Low FLOAT,\
            Close FLOAT,\
            Volume FLOAT,\
            OI FLOAT)"
    cursor.execute(sql)
    
    
    data = pd.read_excel(DATA_DIR+"/"+filename,header=None)
    data = data.fillna("NULL")
    
    
    for i in range(1,len(data)):
        cursor = db.cursor()
        sql = "insert into Iron_original_data." + contract + " values ('"+ data.iloc[i][0].strftime("%Y-%m-%d") + "',"\
                + str(data.iloc[i][1]) + ","\
                + str(data.iloc[i][2]) + ","\
                + str(data.iloc[i][3]) + ","\
                + str(data.iloc[i][4]) + ","\
                + str(data.iloc[i][5]) + ","\
                + str(data.iloc[i][6]) + ")"
        index = i
        cursor.execute(sql)
    cursor.close()

C:\Users\Philip\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: Warning: (1007, "Can't create database 'iron_original_data'; database exists")
  


In [3]:
db = MySQLdb.connect("localhost","root","123456")
db.set_character_set("gbk")
data = pd.read_excel('../all IOE/IOEZ16.xlsx',header=None)
data = data.fillna("NULL")
cursor = db.cursor()
  
sql = "CREATE TABLE IF NOT EXISTS Oil_original_data.IOEZ16 "\
        " (Date DATE,\
        Open FLOAT,\
        High FLOAT,\
        Low FLOAT,\
        Close FLOAT,\
        Volume FLOAT,\
        OI FLOAT)"
cursor.execute(sql)
    
for i in range(1,len(data)):
    cursor = db.cursor()
    sql = "insert into Oil_original_data.ioez16 values ('"+ data.iloc[i][0].strftime("%Y-%m-%d") + "',"\
                + str(data.iloc[i][1]) + ","\
                + str(data.iloc[i][2]) + ","\
                + str(data.iloc[i][3]) + ","\
                + str(data.iloc[i][4]) + ","\
                + str(data.iloc[i][5]) + ","\
                + str(data.iloc[i][6]) + ")"
        
    cursor.execute(sql)
db.commit()

# 合約分開

In [4]:
DATA_DIR = '../ALL CONCRACT/大商所'
for filename in os.listdir(DATA_DIR):
    
    data = pd.read_excel(DATA_DIR + "/" + filename,header=None)
    
    folderName = filename.split(".xlsx")[0]
    
    name_column = 1
    column = 0
    next_column = 8
    while next_column < len(data.loc[0]):
        name = data.loc[0][name_column]
        split_data = data.iloc[8::,column:next_column]

        #刪除所有列都是空值的行
        split_data = split_data.dropna(how='all')

        #strip()刪除字串空白
        head=['Date','PX_OPEN','PX_HIGH','PX_LOW','PX_LAST','PX_SETTLE','VOLUME','OPEN_INT']

        split_data.to_csv('../ALL CONCRACT/DCE/'+folderName+"/"+name.strip()+'.csv',header=head,index=None)
        column += 9
        next_column += 9
        name_column += 9

# 大商所合约

In [131]:
FOLDER_DIR = '../ALL CONCRACT/DCE'
for foldername in os.listdir(FOLDER_DIR):
    cursor = db.cursor()
    
    databaseName = foldername + "_day_data"
    
    sql = "create database if not exists " + databaseName
    cursor.execute(sql)
    
    DATA_DIR = FOLDER_DIR + "/" + foldername
    for filename in os.listdir(DATA_DIR):
        
        name = filename.split('.csv')[0]
        table_name = name.split(' ')[0]
        
        data = pd.read_csv(DATA_DIR + "/" + filename,header=0)
        data['Date'] = pd.to_datetime(data['Date'])

        data = data.where((pd.notnull(data)), None)
        
        sql = "create table if not exists "+ databaseName + "." + table_name + \
                  " (Date DATE PRIMARY KEY,\
                    Open FLOAT(5),\
                    High  FLOAT(5),\
                    Low  FLOAT(5),\
                    Close  FLOAT(5),\
                    Settle  FLOAT(5),\
                    Volume  FLOAT(5),\
                    OI  FLOAT(5))"
        cursor.execute(sql)
        
        if len(data) == 0:
            continue
        
        values = []
        for i in range(len(data)):
            values.append((data.iloc[i][0],data.iloc[i][1],data.iloc[i][2],data.iloc[i][3],data.iloc[i][4],
                           data.iloc[i][5],data.iloc[i][6],data.iloc[i][7]))
        sql = "insert into " + databaseName + "." + table_name + " values (%s,%s,%s,%s,%s,%s,%s,%s)"
        cursor.executemany(sql,values)
        db.commit()   
    cursor.close()

KeyboardInterrupt: 

In [127]:
db.close()